<a href="https://colab.research.google.com/github/Pavith00/job-text-parser/blob/main/job_text_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install spaCy
!pip install -U spacy
!python -m spacy download en_core_web_sm  # Download a pre-trained language model for English


Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 146, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/lib/python3.10/dist-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/usr/local/lib/python3.10/dist-packages/spacy/errors.py", line 3, in <module>
    from .compat import Literal
  File "/usr/local/lib/python3.10/dist-packages/spacy/compat.py", line 4, in <module>
    from thinc.util import copy_array
  File "/usr/local/lib/python3.10/dist-packages/thinc/__init__.py", line 5, in <module>
    from .config import registry
  File "/usr/local/lib/python3.10/dist-packages/thinc/config.py", line 2, in <module>
    import confect

In [ ]:
import pandas as pd
import spacy
from spacy.training.example import Example
from spacy.util import minibatch


In [ ]:

# Mount Google Drive to access your dataset
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:

# Load the CSV file from Google Drive
dataset_path = "/content/drive/My Drive/Cv_project/criteria.csv"
data = pd.read_csv(dataset_path)
data.head()


,criteria,Age Criteria,O/L sitting,O/L total passes,O/L credits,O/L credit subjects,O/L other,O/L other subjects,A/L sitting,A/L total passes,London O/L sitting,London O/L total passes,London O/L credits,London O/L credit subjects,London O/L other,London O/L other subjects,London A/L sitting,London A/L total passes
0,Applicants must be aged 30 or 35 as of the app...,30-35,1,6,4.0,"Maths, Sinhala/Tamil, and English",NaN,NaN,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Applicants aged 30 to 35 are not required to h...,30-35,1,0,0.0,NaN,NaN,NaN,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Applicants must be aged 30 or 35 as of the app...,30-35,1,6,3.0,"Maths, Sinhala/Tamil, and English",NaN,NaN,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Applicants must be aged 30 or 35 as of the app...,30-35,1,6,3.0,any,S or above,"Maths, Sinhala/Tamil, and English",0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Applicants must be aged 30 or 35 as of the app...,30-35,2,6,4.0,any,S or above,"Maths, Sinhala/Tamil, and English",1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import re

# Define field labels in spaCy's format
labels = [
    "AGE_CRITERIA", "OL_SITTING", "OL_TOTAL_PASSES", "OL_CREDITS",
    "OL_CREDIT_SUBJECTS", "OL_OTHER", "OL_OTHER_SUBJECTS",
    "AL_SITTING", "AL_TOTAL_PASSES", "LONDON_OL_SITTING",
    "LONDON_OL_TOTAL_PASSES", "LONDON_OL_CREDITS",
    "LONDON_OL_CREDIT_SUBJECTS", "LONDON_OL_OTHER",
    "LONDON_OL_OTHER_SUBJECTS", "LONDON_AL_SITTING", "LONDON_AL_TOTAL_PASSES"
]

# Process dataset to create training data with labeled entities
def create_training_data(row):
    text = row['criteria']
    entities = []
    for label, col in zip(labels, row.index[1:]):  # Skip 'criteria' column
        value = str(row[col])
        if value and value != 'nan':
            start = text.find(value)
            if start != -1:
                end = start + len(value)
                entities.append((start, end, label))
    return (text, {"entities": entities})

# Create training data in spaCy format
training_data = [create_training_data(row) for _, row in data.iterrows()]
training_data = [d for d in training_data if d[1]["entities"]]  # Remove rows with no entities


In [ ]:


# Define field labels
labels = [
    "AGE_CRITERIA", "OL_SITTING", "OL_TOTAL_PASSES", "OL_CREDITS",
    "OL_CREDIT_SUBJECTS", "OL_OTHER", "OL_OTHER_SUBJECTS",
    "AL_SITTING", "AL_TOTAL_PASSES", "LONDON_OL_SITTING",
    "LONDON_OL_TOTAL_PASSES", "LONDON_OL_CREDITS",
    "LONDON_OL_CREDIT_SUBJECTS", "LONDON_OL_OTHER",
    "LONDON_OL_OTHER_SUBJECTS", "LONDON_AL_SITTING", "LONDON_AL_TOTAL_PASSES"
]

# Function to create training data
def create_training_data(row):
    text = row['criteria']
    entities = []
    seen_ranges = []  # Track seen character ranges to avoid overlaps
    for label, col in zip(labels, row.index[1:]):  # Skip 'criteria' column
        value = str(row[col])
        if value and value != 'nan':
            start = text.find(value)
            if start != -1:
                end = start + len(value)
                # Check for overlap with existing entities
                if not any(start < r[1] and end > r[0] for r in seen_ranges):
                    entities.append((start, end, label))
                    seen_ranges.append((start, end))
    return (text, {"entities": entities})

# Create training data in spaCy format
training_data = [create_training_data(row) for _, row in data.iterrows()]
training_data = [d for d in training_data if d[1]["entities"]]  # Remove rows with no entities

# Initialize a blank spaCy model
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

# Add labels to the NER pipeline
for label in labels:
    ner.add_label(label)

# Train the model
def train_spacy(data, n_iter=30):
    optimizer = nlp.initialize()
    for i in range(n_iter):
        losses = {}
        batches = minibatch(data, size=8)
        for batch in batches:
            examples = [Example.from_dict(nlp.make_doc(text), annots) for text, annots in batch]
            nlp.update(examples, drop=0.2, sgd=optimizer, losses=losses)
        print(f"Iteration {i + 1} - Losses: {losses}")

train_spacy(training_data)

# Save the trained model
output_dir = "/content/drive/My Drive/Cv_project/criteria_ner_model"
nlp.to_disk(output_dir)
print(f"Model saved to {output_dir}")


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Applicants must be aged 30 or 35 as of the applica..." with entities "[(115, 116, 'OL_SITTING'), (103, 104, 'OL_TOTAL_PA...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Applicants aged 30 to 35 are not required to have ..." with entities "[(17, 18, 'OL_TOTAL_PASSES'), (16, 17, 'AL_TOTAL_P...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Iteration 1 - Losses: {'ner': np.float32(3457.7754)}
Iteration 2 - Losses: {'ner': np.float32(494.6275)}
Iteration 3 - Losses: {'ner': np.float32(371.10992)}
Iteration 4 - Losses: {'ner': np.float32(1529.8984)}
Iteration 5 - Losses: {'ner': np.float32(1282.1704)}
Iteration 6 - Losses: {'ner': np.float32(567.39844)}
Iteration 7 - Losses: {'ner': np.float32(828.4882)}
Iteration 8 - Losses: {'ner': np.float32(2060.9424)}
Iteration 9 - Losses: {'ner': np.float32(641.3766)}
Iteration 10 - Losses: {'ner': np.float32(921.0154)}
Iteration 11 - Losses: {'ner': np.float32(727.15936)}
Iteration 12 - Losses: {'ner': np.float32(809.5624)}
Iteration 13 - Losses: {'ner': np.float32(392.08017)}
Iteration 14 - Losses: {'ner': np.float32(265.7971)}
Iteration 15 - Losses: {'ner': np.float32(189.22807)}
Iteration 16 - Losses: {'ner': np.float32(175.05627)}
Iteration 17 - Losses: {'ner': np.float32(162.12349)}
Iteration 18 - Losses: {'ner': np.float32(172.54654)}
Iteration 19 - Losses: {'ner': np.float32(1

In [ ]:
# Load the trained model
nlp = spacy.load(output_dir)

# Test with sample input
test_text = "Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 5 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, and A/L qualifications requiring 3 passes in 1 sitting."
doc = nlp(test_text)-

for ent in doc.ents:
    print(ent.text, ent.label_)


30 AGE_CRITERIA
5 OL_TOTAL_PASSES
1 OL_SITTING
Maths, Sinhala/Tamil, and English OL_CREDIT_SUBJECTS


In [ ]:
import pandas as pd
import spacy
from spacy.training.example import Example
from spacy.util import minibatch

# Load the CSV file from Google Drive
dataset_path = "/content/drive/My Drive/Cv_project/criteria.csv"
data = pd.read_csv(dataset_path)

# Define field labels
labels = [
    "AGE_CRITERIA", "OL_SITTING", "OL_TOTAL_PASSES", "OL_CREDITS",
    "OL_CREDIT_SUBJECTS", "OL_OTHER", "OL_OTHER_SUBJECTS",
    "AL_SITTING", "AL_TOTAL_PASSES", "LONDON_OL_SITTING",
    "LONDON_OL_TOTAL_PASSES", "LONDON_OL_CREDITS",
    "LONDON_OL_CREDIT_SUBJECTS", "LONDON_OL_OTHER",
    "LONDON_OL_OTHER_SUBJECTS", "LONDON_AL_SITTING", "LONDON_AL_TOTAL_PASSES"
]

# Function to create training data
def create_training_data(row):
    text = row['criteria']
    entities = []
    seen_ranges = []  # Track seen character ranges to avoid overlaps
    for label, col in zip(labels, row.index[1:]):  # Skip 'criteria' column
        value = str(row[col])
        if value and value.lower() != 'nan':  # Ensure we don't consider 'nan'
            start = text.lower().find(value.lower())  # Case insensitive search
            while start != -1:  # Handle multiple occurrences of the same value
                end = start + len(value)
                # Check for overlap with existing entities
                if not any(start < r[1] and end > r[0] for r in seen_ranges):
                    entities.append((start, end, label))
                    seen_ranges.append((start, end))
                start = text.lower().find(value.lower(), end)  # Find next occurrence
    return (text, {"entities": entities})

# Create training data in spaCy format
training_data = [create_training_data(row) for _, row in data.iterrows()]
training_data = [d for d in training_data if d[1]["entities"]]  # Remove rows with no entities

# Initialize a blank spaCy model
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

# Add labels to the NER pipeline
for label in labels:
    ner.add_label(label)

# Train the model
def train_spacy(data, n_iter=50):  # Increased iterations for better learning
    optimizer = nlp.initialize()
    for i in range(n_iter):
        losses = {}
        batches = minibatch(data, size=8)
        for batch in batches:
            examples = [Example.from_dict(nlp.make_doc(text), annots) for text, annots in batch]
            nlp.update(examples, drop=0.2, sgd=optimizer, losses=losses)
        print(f"Iteration {i + 1} - Losses: {losses}")

train_spacy(training_data)

# Save the trained model
output_dir = "/content/drive/My Drive/Cv_project/criteria_ner_model"
nlp.to_disk(output_dir)
print(f"Model saved to {output_dir}")

# Load the trained model for testing
nlp = spacy.load(output_dir)

# Test with the complex input sentence
test_text = "Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L."

print(f"Testing on: {test_text}")
doc = nlp(test_text)

# Print extracted entities
for ent in doc.ents:
    print(ent.text, ent.label_)



/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Applicants must be aged 30 or 35 as of the applica..." with entities "[(115, 116, 'OL_SITTING'), (235, 236, 'OL_SITTING'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Applicants aged 30 to 35 are not required to have ..." with entities "[(17, 18, 'OL_TOTAL_PASSES'), (16, 17, 'AL_TOTAL_P...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Iteration 1 - Losses: {'ner': np.float32(3358.7068)}
Iteration 2 - Losses: {'ner': np.float32(701.84174)}
Iteration 3 - Losses: {'ner': np.float32(433.6249)}
Iteration 4 - Losses: {'ner': np.float32(1377.9619)}
Iteration 5 - Losses: {'ner': np.float32(423.75778)}
Iteration 6 - Losses: {'ner': np.float32(799.20325)}
Iteration 7 - Losses: {'ner': np.float32(491.6631)}
Iteration 8 - Losses: {'ner': np.float32(1118.5846)}
Iteration 9 - Losses: {'ner': np.float32(379.79047)}
Iteration 10 - Losses: {'ner': np.float32(415.80972)}
Iteration 11 - Losses: {'ner': np.float32(224.63307)}
Iteration 12 - Losses: {'ner': np.float32(185.6026)}
Iteration 13 - Losses: {'ner': np.float32(200.0529)}
Iteration 14 - Losses: {'ner': np.float32(146.9573)}
Iteration 15 - Losses: {'ner': np.float32(158.65207)}
Iteration 16 - Losses: {'ner': np.float32(146.8447)}
Iteration 17 - Losses: {'ner': np.float32(140.53012)}
Iteration 18 - Losses: {'ner': np.float32(143.11618)}
Iteration 19 - Losses: {'ner': np.float32(1

In [ ]:
import pandas as pd
import spacy
from spacy.training.example import Example
from spacy.util import minibatch

# Load the CSV file from Google Drive
dataset_path = "/content/drive/My Drive/Cv_project/criteria.csv"
data = pd.read_csv(dataset_path)

# Define field labels
labels = [
    "AGE_CRITERIA", "OL_SITTING", "OL_TOTAL_PASSES", "OL_CREDITS",
    "OL_CREDIT_SUBJECTS", "OL_OTHER", "OL_OTHER_SUBJECTS",
    "AL_SITTING", "AL_TOTAL_PASSES", "LONDON_OL_SITTING",
    "LONDON_OL_TOTAL_PASSES", "LONDON_OL_CREDITS",
    "LONDON_OL_CREDIT_SUBJECTS", "LONDON_OL_OTHER",
    "LONDON_OL_OTHER_SUBJECTS", "LONDON_AL_SITTING", "LONDON_AL_TOTAL_PASSES"
]

# Function to create training data
def create_training_data(row):
    text = row['criteria']
    entities = []
    seen_ranges = []  # Track seen character ranges to avoid overlaps
    for label, col in zip(labels, row.index[1:]):  # Skip 'criteria' column
        value = str(row[col])
        if value and value.lower() != 'nan':  # Ensure we don't consider 'nan'
            start = text.lower().find(value.lower())  # Case insensitive search
            while start != -1:  # Handle multiple occurrences of the same value
                end = start + len(value)
                # Check for overlap with existing entities
                if not any(start < r[1] and end > r[0] for r in seen_ranges):
                    entities.append((start, end, label))
                    seen_ranges.append((start, end))
                start = text.lower().find(value.lower(), end)  # Find next occurrence
    return (text, {"entities": entities})

# Create training data in spaCy format
training_data = [create_training_data(row) for _, row in data.iterrows()]
training_data = [d for d in training_data if d[1]["entities"]]  # Remove rows with no entities

# Initialize a blank spaCy model
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

# Add labels to the NER pipeline
for label in labels:
    ner.add_label(label)

# Train the model
def train_spacy(data, n_iter=50):  # Increased iterations for better learning
    optimizer = nlp.initialize()
    for i in range(n_iter):
        losses = {}
        batches = minibatch(data, size=8)
        for batch in batches:
            examples = [Example.from_dict(nlp.make_doc(text), annots) for text, annots in batch]
            nlp.update(examples, drop=0.2, sgd=optimizer, losses=losses)
        print(f"Iteration {i + 1} - Losses: {losses}")

train_spacy(training_data)

# Save the trained model
output_dir = "/content/drive/My Drive/Cv_project/criteria_ner_model"
nlp.to_disk(output_dir)
print(f"Model saved to {output_dir}")

# Load the trained model for testing
nlp = spacy.load(output_dir)

# Test with the complex input sentence
test_text = "Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L."

print(f"Testing on: {test_text}")
doc = nlp(test_text)

# Print extracted entities
for ent in doc.ents:
    print(ent.text, ent.label_)



/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Applicants must be aged 30 or 35 as of the applica..." with entities "[(115, 116, 'OL_SITTING'), (235, 236, 'OL_SITTING'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Applicants aged 30 to 35 are not required to have ..." with entities "[(17, 18, 'OL_TOTAL_PASSES'), (16, 17, 'AL_TOTAL_P...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Iteration 1 - Losses: {'ner': 3415.530641892685}
Iteration 2 - Losses: {'ner': 696.1529139666792}
Iteration 3 - Losses: {'ner': 543.3985685478208}
Iteration 4 - Losses: {'ner': 1436.9439250499563}
Iteration 5 - Losses: {'ner': 526.5817345911717}
Iteration 6 - Losses: {'ner': 654.6991545780677}
Iteration 7 - Losses: {'ner': 1177.1472971133078}
Iteration 8 - Losses: {'ner': 485.7879115081872}
Iteration 9 - Losses: {'ner': 510.0877263755466}
Iteration 10 - Losses: {'ner': 335.6875946753561}
Iteration 11 - Losses: {'ner': 1003.0898038379079}
Iteration 12 - Losses: {'ner': 334.66091100122713}
Iteration 13 - Losses: {'ner': 171.61718818503095}
Iteration 14 - Losses: {'ner': 177.11903680146517}
Iteration 15 - Losses: {'ner': 150.34902457754734}
Iteration 16 - Losses: {'ner': 146.67829025593997}
Iteration 17 - Losses: {'ner': 139.8309022499919}
Iteration 18 - Losses: {'ner': 140.3105589644804}
Iteration 19 - Losses: {'ner': 128.92547498493778}
Iteration 20 - Losses: {'ner': 141.56791721946996}

In [ ]:
output_dir = "/content/drive/My Drive/Cv_project/criteria_ner_model"
# Load the trained model for testing
nlp = spacy.load(output_dir)

# Test with the complex input sentence
test_text = "Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L."

print(f"Testing on: {test_text}")
doc = nlp(test_text)

# Print extracted entities
for ent in doc.ents:
    print(ent.text, ent.label_)



Testing on: Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L.
6 OL_TOTAL_PASSES
1 OL_SITTING
Maths, Sinhala/Tamil, and English OL_CREDIT_SUBJECTS
6 OL_TOTAL_PASSES
1 OL_SITTING
3 AL_TOTAL_PASSES
1 OL_SITTING


In [ ]:
# Load the trained model for testing
nlp = spacy.load(output_dir)

# Test with the complex input sentence
test_text = " Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L."

print(f"Testing on: {test_text}")
doc = nlp(test_text)

# Print extracted entities and model state
print("Extracted Entities:")
for ent in doc.ents:
    print(ent.text, ent.label_)

# Print available labels in the model
print("Available labels in the model:")
print(nlp.get_pipe("ner").labels)


Testing on:  Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L.
Extracted Entities:
6 OL_TOTAL_PASSES
1 OL_SITTING
Maths, Sinhala/Tamil, and English OL_CREDIT_SUBJECTS
6 OL_TOTAL_PASSES
1 OL_SITTING
3 AL_TOTAL_PASSES
1 OL_SITTING
Available labels in the model:
('AGE_CRITERIA', 'AL_SITTING', 'AL_TOTAL_PASSES', 'LONDON_AL_SITTING', 'LONDON_AL_TOTAL_PASSES', 'LONDON_OL_CREDITS', 'LONDON_OL_CREDIT_SUBJECTS', 'LONDON_OL_OTHER', 'LONDON_OL_OTHER_SUBJECTS', 'LONDON_OL_SITTING', 'LONDON_OL_TOTAL_PASSES', 'OL_CREDITS', 'OL_CREDIT_SUBJECTS', 'OL_OTHER', 'OL_OTHER_SUBJECTS', 'OL_SITTING', 'OL_TOTAL_PASSES')


In [ ]:
# Load the trained model for testing
nlp = spacy.load(output_dir)

# Test with the complex input sentence
test_text = "Applicants must be aged 31 or 33 as of the application closing date, with O/L qualifications requiring 5 passes in 1 sitting, including 2 credit passes in any subjects. Maths, Chemistry, and English needing a minimum grade of S or above, and A/L qualifications requiring 3 passes in 1 sitting."

print(f"Testing on: {test_text}")
doc = nlp(test_text)

# Print extracted entities
for ent in doc.ents:
    print(ent.text, ent.label_)


Testing on: Applicants must be aged 31 or 33 as of the application closing date, with O/L qualifications requiring 5 passes in 1 sitting, including 2 credit passes in any subjects. Maths, Chemistry, and English needing a minimum grade of S or above, and A/L qualifications requiring 3 passes in 1 sitting.
5 OL_TOTAL_PASSES
1 OL_SITTING
any OL_CREDIT_SUBJECTS
Maths, Chemistry, and English OL_OTHER_SUBJECTS
S or above OL_OTHER
3 AL_TOTAL_PASSES
1 OL_SITTING


In [ ]:
# Train the model with increased iterations
def train_spacy(data, n_iter=100):  # Increased iterations for better learning
    optimizer = nlp.initialize()
    for i in range(n_iter):
        losses = {}
        batches = minibatch(data, size=8)  # Experiment with batch sizes
        for batch in batches:
            examples = [Example.from_dict(nlp.make_doc(text), annots) for text, annots in batch]
            nlp.update(examples, drop=0.5, sgd=optimizer, losses=losses)  # Increased dropout
        print(f"Iteration {i + 1} - Losses: {losses}")

# Re-train with updated parameters
train_spacy(training_data)

# Test the model with known input
test_text = "Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L."
doc = nlp(test_text)

# Check the output
for ent in doc.ents:
    print(ent.text, ent.label_)


Iteration 1 - Losses: {'ner': np.float32(3521.713)}
Iteration 2 - Losses: {'ner': np.float32(625.3037)}
Iteration 3 - Losses: {'ner': np.float32(531.921)}
Iteration 4 - Losses: {'ner': np.float32(451.07037)}
Iteration 5 - Losses: {'ner': np.float32(419.28888)}
Iteration 6 - Losses: {'ner': np.float32(510.9321)}
Iteration 7 - Losses: {'ner': np.float32(834.10944)}
Iteration 8 - Losses: {'ner': np.float32(614.5972)}
Iteration 9 - Losses: {'ner': np.float32(567.4642)}
Iteration 10 - Losses: {'ner': np.float32(758.2608)}
Iteration 11 - Losses: {'ner': np.float32(869.0703)}
Iteration 12 - Losses: {'ner': np.float32(526.6804)}
Iteration 13 - Losses: {'ner': np.float32(544.0411)}
Iteration 14 - Losses: {'ner': np.float32(530.759)}
Iteration 15 - Losses: {'ner': np.float32(322.54758)}
Iteration 16 - Losses: {'ner': np.float32(300.75345)}
Iteration 17 - Losses: {'ner': np.float32(327.89316)}
Iteration 18 - Losses: {'ner': np.float32(248.24971)}
Iteration 19 - Losses: {'ner': np.float32(259.308)

Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L."


In [ ]:
# Test the model with known input
test_text = "Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L."
doc = nlp(test_text)

# Check the output
for ent in doc.ents:
    print(ent.text, ent.label_)

30 AGE_CRITERIA
6 OL_TOTAL_PASSES
1 OL_SITTING
Maths, Sinhala/Tamil, and English OL_CREDIT_SUBJECTS


In [ ]:
# Ensure you have a consistent approach to case sensitivity
def create_training_data(row):
    text = row['criteria'].lower()  # Convert to lower case
    entities = []
    seen_ranges = []  # Track seen character ranges to avoid overlaps
    for label, col in zip(labels, row.index[1:]):  # Skip 'criteria' column
        value = str(row[col]).lower()  # Lowercase value for consistency
        if value and value != 'nan':
            start = text.find(value)
            while start != -1:
                end = start + len(value)
                if not any(start < r[1] and end > r[0] for r in seen_ranges):
                    entities.append((start, end, label))
                    seen_ranges.append((start, end))
                start = text.find(value, end)
    return (text, {"entities": entities})

# Modify the training and testing process as needed
train_spacy(training_data)

# Test with a known input
test_text = "Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L."
test_text_lower = test_text.lower()  # Convert test text to lower case
doc = nlp(test_text_lower)

# Check the output
print("Entities found:")
for ent in doc.ents:
    print(f"Text: {ent.text}, Label: {ent.label_}")


Iteration 1 - Losses: {'ner': np.float32(3573.1985)}
Iteration 2 - Losses: {'ner': np.float32(619.22565)}
Iteration 3 - Losses: {'ner': np.float32(540.54816)}
Iteration 4 - Losses: {'ner': np.float32(430.8973)}
Iteration 5 - Losses: {'ner': np.float32(375.99622)}
Iteration 6 - Losses: {'ner': np.float32(514.81366)}
Iteration 7 - Losses: {'ner': np.float32(721.1455)}
Iteration 8 - Losses: {'ner': np.float32(430.10947)}
Iteration 9 - Losses: {'ner': np.float32(641.17896)}
Iteration 10 - Losses: {'ner': np.float32(552.4101)}
Iteration 11 - Losses: {'ner': np.float32(870.27057)}
Iteration 12 - Losses: {'ner': np.float32(858.9715)}
Iteration 13 - Losses: {'ner': np.float32(508.28046)}
Iteration 14 - Losses: {'ner': np.float32(804.7556)}
Iteration 15 - Losses: {'ner': np.float32(706.08167)}
Iteration 16 - Losses: {'ner': np.float32(454.19687)}
Iteration 17 - Losses: {'ner': np.float32(1377.2225)}
Iteration 18 - Losses: {'ner': np.float32(539.4405)}
Iteration 19 - Losses: {'ner': np.float32(5

In [ ]:


# Load the trained model for testing
nlp = spacy.load(output_dir)

# Test with the complex input sentence
test_text = "Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L."

print(f"Testing on: {test_text}")
doc = nlp(test_text)

# Print extracted entities
for ent in doc.ents:
    print(ent.text, ent.label_)

Testing on: Applicants must be aged 30 or 35 as of the application closing date, with O/L qualifications requiring 6 passes in 1 sitting, including 4 credit passes in Maths, Sinhala/Tamil, and English, or alternatively, 6 passes in 1 sitting from London O/L with 4 credits including Mathematics and English; A/L qualifications require 3 passes in 1 sitting from either GCE A/L or London A/L.
30 AGE_CRITERIA
6 OL_TOTAL_PASSES
1 OL_SITTING
Maths, Sinhala/Tamil, and English OL_CREDIT_SUBJECTS
